## Read encoded track data and decode it
- Disc read data will invert on every read pulse (0->1, 1->0)
- This data represents how long the same bit value continued (bit interval, 000 -> 3, 11111 -> 5)
- The bit interval is encoded with `ord(' ')+bit_interval`. 
```sh
FDD output    :  111101110111110111011101
Captured data :  000011110000001111000011
This data     :     4   4     6   4   4 2
```

In [1]:
test = True

In [3]:
#track_data = 'track1-4m-a.txt'
track_data = 'track2e-4m-a.txt'

with open(track_data, 'r') as f:
    for line in f:
        line = line.rstrip('\n')
        if '**TRACK_READ' in line:
            str = ''
        elif '**TRACK_END' in line:
            pass
        else:
            str += line

interval_buf = [ ord(c)-ord(' ') for c in str ]

if test:
    print(str)

,/1/1///001/0//01000//00000//000000/0/100/0/00000//0001/0.00001///00100//000000//0001/0/0/1/1/0/00000//0001///10000//000000.00100/0.00100///10000.00001/0.001/1.00000/1.000000//100/1.000000//00100///01000//0001/1./01000//0/101///001/1.00000/0/00000/0/00000/0/100/0/00000/0/001/0/0/100///10000.0/100/0/00000/0/001/0//0010///1/100//000000.0000000.001/0/0/001/0.00100//000000//0001/0.001/1.0/1/1/0/0/100//000000.00000/0/00000/0/001/0.00001///1/10//0/100/0/00000//00000//00000/0/001/0/0/100/0/00000//00000//00000/0/00000//01/00//00100.0/100/0/00000//000000//0000@0/0.98@///001/7-/89/07879//8879.09697/09697/09787/17887007887007969/07969/07969//8888//9788.09787/09787/18796007896006988/07969//8969//8879./9788//96970/9787/16897006978/07969/07879./9788//9697/08797/0788700696900696://7879//8789.08698//9697007887/169780069690/7879//8879/.:689./:688/08787008787007887007879/07969//8969./9879./:0/0/000/1//00000//00000/0/001/1.0/1/1/0/00000//0001/0//0100/0/00000//000000/0/1/1/0/00000//00008A6@80?8@7/@7@90.0/

## Display histogram of bit interval
In case the reading clock is 4MHz, 1 represents 0.25us

In [4]:
def generate_histogram(data):
    histo = [ 0 ] * 256
    for l in data:
        if l>=len(histo):
            l = len(histo) - 1
        histo[l]+=1
    return histo

if test:
    histo = generate_histogram(interval_buf)
    for i,v in enumerate(histo):
        print(i, v)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 2
11 6
12 9
13 11
14 1216
15 10190
16 23956
17 4560
18 2
19 4
20 1
21 12
22 581
23 1732
24 1834
25 2044
26 427
27 4
28 1
29 1
30 11
31 82
32 108
33 59
34 25
35 4
36 0
37 0
38 0
39 1
40 1
41 0
42 0
43 0
44 1
45 1
46 2
47 0
48 0
49 1
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
1

## Otsu thresholding method

In [6]:
import math
import numpy as np

class Otsu:
    def __init__(self):
        self.threshold_values = {}
        self.histo = []

    def generate_histogram(self, data):
        self.histo = np.zeros(256)
        for i in data:
            if i>255:
                continue
            self.histo[i] += 1
        return self.histo

    def display_histogram(self):
        for i,h in enumerate(self.histo):
            print(i, " : ", h)

    def countData(self):
        cnt = 0
        for i in range(0, len(self.histo)):
            if self.histo[i]>0:
                cnt += self.histo[i]
        return cnt

    def wieght(self, s, e):
        w = 0
        for i in range(s, e):
            w += self.histo[i]
        return w

    def mean(self, s, e):
        m = 0
        w = self.wieght(s, e)
        for i in range(s, e):
            m += self.histo[i] * i
        if w==0:
            return 0
        return m/float(w)

    def variance(self, s, e):
        v = 0
        m = self.mean(s, e)
        w = self.wieght(s, e)
        for i in range(s, e):
            v += ((i - m) **2) * self.histo[i]
        if w==0:
            return 0
        return v/w
                
    def threshold(self):
        self.threshold_values = {}      # Clear threshold value dict
        cnt = self.countData()
        for i in range(1, len(self.histo)):
            vb = self.variance(0, i)
            wb = self.wieght(0, i) / float(cnt)
            mb = self.mean(0, i)
            
            vf = self.variance(i, len(self.histo))
            wf = self.wieght(i, len(self.histo)) / float(cnt)
            mf = self.mean(i, len(self.histo))
            
            V2w = wb * (vb) + wf * (vf)
            V2b = wb * wf * (mb - mf)**2
            
            if not math.isnan(V2w):
                self.threshold_values[i] = V2w

    def mask_histogram(self, s, e):
        for i in range(s, e+1):
            self.histo[i] = 0

    def get_optimal_threshold(self):
        min_V2w = min(self.threshold_values.values())
        optimal_threshold = [k for k,v in self.threshold_values.items() if v == min_V2w]
        return optimal_threshold[0]

## Using Otsu-thresholding-method to identify the threshold value for data-cell separation
2D/2DD bit period patterns = 4us / 6us / 8us

In [7]:
if test:
    otsu = Otsu()
    otsu.generate_histogram(interval_buf)

    otsu.threshold()
    th1 = otsu.get_optimal_threshold()

    otsu.mask_histogram(0, th1)
    otsu.threshold()
    th2 = otsu.get_optimal_threshold()

    print(th1, th2)

21 29


## C/D (Clock/Data) pattern restoration
Each 1s and 0s in this data represents 2us time period

In [27]:
class data_separator:
    def __init__(self, clk_spd = 1e6, tracking_rate=0.01):
        self.clock_speed   = clk_spd
        self.bit_cell      = 500e3    # 1/2MHz
        self.cell_size     = self.clock_speed / self.bit_cell
        self.cell_size_ref = self.cell_size
        self.cell_size_max = self.cell_size * 1.5
        self.cell_size_min = self.cell_size * 0.5
        self.tracking_rate = tracking_rate
        #print(self.cell_size, self.cell_size_min, self.cell_size_max)

    # Review entire interval_buf and get the most frequent interval.
    # Calculate the cell_size based on the most frequent interval
    # Assuming the most frequent interval = 2 bit cell size (the most short interval)
    def review(self, interval_buf):
        cell_size = self.cell_size_ref
        histo = [ 0 ] * 256
        # Generate a histogram
        for interval in interval_buf:
            if interval > cell_size * 5:                   # ignore a pulse with too long pulse interval
                continue
            histo[interval] += 1
        max_    = 0
        max_idx = -1
        # Get the maximum value
        for idx, h in enumerate(histo):
            #print(idx, ' ', h)
            if h > max_:
                max_ = h
                max_idx = idx
        max_idx /= 2                                       # assuming the most frequent interval is cell_size * 2
        max_idx = max(max_idx, self.cell_size_min)
        max_idx = min(max_idx, self.cell_size_max)
        self.cell_size = max_idx
        return max_idx

    def pulse(self, interval):                             # interval = pulse interval in 'cell_size' unit
        #print(interval, ' ', end='')
        interval_buf = int(interval / self.cell_size + 0.5)
        error = interval - interval_buf * self.cell_size
        #print(interval_buf, self.cell_size, error)
        self.cell_size += error * self.tracking_rate       # Correct the cell_size
        # cell size range limitter
        self.cell_size = max(self.cell_size, self.cell_size_min)
        self.cell_size = min(self.cell_size, self.cell_size_max)
        return interval_buf


def restoreCD(interval_buf_buf, th1, th2):
    stream = ''
    for lb in interval_buf_buf:
        if   lb <= th1:
            stream +=   '01'
        elif lb <= th2:
            stream +=  '001'
        else:
            stream += '0001'
    return stream

def restoreCD2(interval_buf_buf, clk_spd = 1e6, tracking_rate = 0.01):
    ds = data_separator(clk_spd, tracking_rate)
    # warm up the data separator and synchronize to the data speed
    #ds.review(interval_buf)
    for lb in interval_buf_buf:
        ds.pulse(lb)
    stream = ''
    for lb in interval_buf_buf:
        num = ds.pulse(lb)
        #print(num, ' ', end='')
        if   num <= 2:
            stream +=   '01'
        elif num == 3:
            stream +=  '001'
        elif num == 4:
            stream += '0001'
        else:
            pass
    return stream



if test:
    stream = restoreCD2(interval_buf[:], 4e6)
    print(stream)

...


## Performs MFM decoding

In [28]:
def decodeMFM(CDstream):
    read_bit_count = 0
    read_data = 0
    raw_bit_stream = 0
    data_valid = False
    missing_clock = False
    MFM_count = 0

    mfm_buf = []  # MFM decoded data buffer
    mc_buf = []   # Missing clock flag buffer

    for c in CDstream:
        bit = 1 if c=='1' else 0
        if read_bit_count % 2==1:
            read_data  = read_data<<1 | bit
        raw_bit_stream = (raw_bit_stream<<1 | bit) & 0x7fff
        read_bit_count += 1

        if read_bit_count >= 16:
            data_valid    = True

        if raw_bit_stream==0x5224 or raw_bit_stream==0x4489:
            data_valid    = True
            missing_clock = True

        if data_valid == True:
            MFM_count += 1
            mfm_buf.append(read_data)
            mc_buf.append(missing_clock)
            read_data      = 0
            read_bit_count = 0
            data_valid     = False
            missing_clock  = False

    return mfm_buf, mc_buf

def dumpMFM(mfm_buf, mc_buf):
    count = 0
    for mfm, mc in zip(mfm_buf, mc_buf):
        if mc==True:
            print('*', end='')
        else:
            print(' ', end='')
        print(format(mfm, '02X'), end='')
        count+=1
        if count==32:
            count=0
            print('')

if test:
    mfm_buf, mc_buf = decodeMFM(stream)
    dumpMFM(mfm_buf, mc_buf)

 FF FF FF FF FF FF FF FF FF*42*02*A1*A1 FE 14 00 01 01 2B 5A 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E
 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 00 00 00 00 00 00 00 00 00 00 00 00 00*14*02*A1*A1 FB 01 85 71 01
 80 6F 01 7B 70 01 79 74 01 7B 77 01 80 78 01 82 79 FF FE 01 30 69 01 2C 68 01 27 68 01 21 68 01
 1F 6A 01 20 6C 01 24 6E 01 28 6E FF FE 01 45 6B 01 41 6C 01 3E 6D 01 3E 6F 01 41 71 01 43 72 FF
 FF FF FE 00 FF 4A 01 08 46 01 10 42 01 15 43 01 18 49 FF FE 01 54 55 01 5B 51 01 5E 56 FF FE 01
 29 7B 01 28 7D 01 25 7D 01 21 7D 01 1E 7D 01 1B 7D 01 1B 7F 01 24 80 FF FE 00 ED 87 00 F9 88 01
 0B 8A 01 17 8A 01 22 8A FF FE 01 7D 84 01 85 85 01 8E 85 01 92 84 FF FE 01 74 60 01 83 5F 01 92
 5E 01 9E 5E FF FE 01 99 5E 01 A2 58 01 A9 52 01 B1*04 00 00 1F C0 31 9F C0 11 FF C0 10 3F C1 00
 4F C1 90 0F C0 00 0F C0 11 FF C0 11 0F C0 00 4F C0 40 CF C0 10 7F C0 90 0F C0 87 0F C0 C6 0F C0
 06 3F C0 06 0F C0 46 0F C1 86 0F C0 06 0F C0 86 00 00 1F C7 80 4F C7 C0 4F C0 30 4F C3 F8*02 00
 24 24 24 24 24 24 24 24 24 24

 33 F0 01 33 03 88 04 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4
 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E4 E0 00 00 00 00 00 00
 00 00 00 00 00 0A*00*02*A1*A1 FE 14 00 09 01 A2 F3 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E
 4E 4E 4E 4E 4E 4E 4E 00 00 00 00 00 00 00 00 00 00 00 00 00 14*00*02*A1*A1 FB 01 F0 1C 01 A3 1B
 01 6A 18 01 3E 18 01 07 19 00 D4 1B 00 9C 1D 00 79 1A FF FF FF FE 00*14 00 1F*C2 60 3F C3 E1 9F
 C8 04 FF C3 90 1F C0 11 1F C0 30 3F C4 10 3F C3 90 FF C0 70*02 7E 67 C0 7E 4C 40 FE 41 C0 7E 1F
 C0 FE 00 49 FE 07 C9 FE 3C 48 7E 30 40 7E 20 40 7E 0E 40 7E 08 40 7E 08 18 7E 0F 90 00 00 FE 60
 38 7E 60 33 FE 48 30 7E 47 30 7E 40 26 7E 40 30 FE 1E 32 7E 19 38 7E*02 07 0F*C2 06 0F C0 26 0F
 C1 06 0F C0 06 3F C0 64 FF C1 C4 0F C0 24 4F C0 44 20 00 1F C0 04 3F C0 04 7F C0 C4 9F C0 06 3F
 C1 86 1F C1 C7 1F C0 C7 3F C0 07 CF C0 30 0F C3 F0 3F C0 00 3F C0 00 0F C0 F1 CF C0 30 3F C1 80
 CF C1 F2 00 00 1F C0 06 3F C0

 60 00 1C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C 9C
 9C 84 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24
 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 24 3F FF FF FF FF FF FF FF FF FF FF FF C5*00*C2
*C2 FC 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E
 4E 4E 4E 4E 4E 4E 4E 4E 4E 4E

## CRC calculation routine
CRC polynomial = CCITT-16  G(X) = 1+X~5+X~12+X~16 == 1 001 000 0010 001 == 0x11021  
FDC calculates CRC from the top of the address marks (0xA1) to CRC values.  
When FDC generates CRC, the CRC field (2 bytes) are set to 0x00,0x00 and the initial CRC value is 0x84Cf.  
However, the top of address marks can't be read correctly when reading the data from the disc. I decided to skip 1st x3 0xA1 values and start CRC caluculation from (F8/FB/FC/FE) with initial value of 0xe59a.

If the data are read correctly, the return value will be 0x0000.
```python
data=[0xfe, 0x01, 0x01, 0x03, 0x01, 0xdd, 0xea]
crc_value = crc.data(data[:])  # crc_value must be 0
```

In [21]:
# G(X) = 1+X~5+X~12+X~16

# 00*14*02*A1*A1 FE 01 01 03 01 DD EA
class CCITT_CRC:
    def __init__(self):
        self.polynomial = 0x1102100    # 0001 0001 0000 0010 0001 0000 0000
        self.reset()

    def reset(self):
        self.crcval = 0xe59a00         # In case the 1st x3 A1s are omitted ( F8/FB/FC/FE )

    def reset2(self):
        self.crcval = 0x84cf00        # In case AM = A1 A1 A1 + F8/FB/FC/FE

    def get(self):
        return (self.crcval>>8) & 0xffff

    def data(self, buf):
        for byte in buf:
            #print(format(byte, '02X'))
            self.crcval  |= byte
            for i in range(8):
                self.crcval <<= 1
                if self.crcval & 0x1000000 != 0:
                    self.crcval  ^= self.polynomial
                   

if test:
    # test
    crc = CCITT_CRC()

    # FE 01 01 03 01 DD EA
    crc.reset()
    data=[0xfe, 0x01, 0x01, 0x03, 0x01, 0xdd, 0xea]
    crc.data(data[:])
    print(format(crc.get(), '04X'))

    # FE 01 01 10 01 8B CA
    crc.reset()
    data=[0xfe, 0x01, 0x01, 0x10, 0x01, 0x8b, 0xca]
    crc.data(data[:])
    print(format(crc.get(), '04X'))

    # FB + FF*256 + FB E5 (FE)
    crc.reset()
    a  = [ 0xfb ]
    a += [ 0xff ] * 256
    a += [ 0xfb, 0xe5 ]
    crc.data(a)
    print(format(crc.get(), '04X'))

    crc.reset()
    crc.data([0xa1, 0xa1, 0xa1])
    print(format(crc.get(), '02X'))


0000
0000
0000
B88B


## Decode IBM format

In [37]:
from enum import Enum

def dump_list_hex(lst):
    print('[ ', end='')
    for i in lst:
        print('0x{}, '.format(format(i, '02X')) , end='')
    print(']')

def decodeFormat(mfm_buf, mc_buf, verbose=False):
    class State(Enum):
        IDLE       = 0
        CHECK_MARK = 1
        INDEX      = 2
        IDAM       = 3
        DAM        = 4
        DDAM       = 5

    state = State.IDLE
    count = 0
    id = []
    sector = []
    track = []
    for dt, mc in zip(mfm_buf, mc_buf):
        if state == State.IDLE:
            if mc == True:        # found a missing clock
                state = State.CHECK_MARK

        elif state == State.CHECK_MARK:
            if mc == True:
                continue
            elif dt == 0xfc:      # Index AM
                state = State.INDEX
            elif dt == 0xfe:      # ID AM
                state = State.IDAM
            elif dt == 0xfb:      # Data AM
                state = State.DAM
            elif dt == 0xf8:      # Deleted Data AM
                state = State.DDAM
            else:
                state = State.IDLE

        elif state == State.INDEX:
            if verbose:
                print('INDEX')
            state = State.IDLE
 
        elif state == State.IDAM:
            if count == 0:
                if verbose:
                    print('IDAM ', end='')
                id = [ 0xfe ]
                count = 4+2   # ID+CRC
            id.append(dt)
            count -= 1
            if count == 0:
                crc.reset()
                crc.data(id)
                id_ = id.copy()
                id = id[1:-2]   # remove IDAM and CRC
                if crc.get()==0:
                    if verbose:
                        print("CRC - OK")
                        print("CHRN=", id)
                else:
                    if verbose:
                        print("CRC - ERROR")
                        dump_list_hex(id_)
                state = State.IDLE

        elif state == State.DAM:
            if len(id)<4:
                print("ERROR - ID=", id)
                state = State.IDLE
                continue
            if count == 0:
                if verbose:
                    print('DAM ', end='')
                sector = [ 0xfb ]
                count = [128, 256, 512, 1024][id[3] & 0x03]
                count += 2   # for CRC
            sector.append(dt)
            count -= 1
            if count == 0:
                crc.reset()
                crc.data(sector)
                _sector = sector.copy()
                sector = sector[1:-2]   # remove DAM and CRC
                if crc.get()==0:
                    if verbose:
                        print("CRC - OK")
                        dump_list_hex(_sector)
                    track.append([id, True, sector])
                else:
                    if verbose:
                        print("CRC - ERROR")
                        dump_list_hex(_sector)
                id=[]
                state = State.IDLE

        elif state == State.DDAM:
            if len(id)<4:
                print("ERROR - ID=", id)
                state = State.IDLE
                continue
            if count == 0:
                if verbose:
                    print('DDAM ', end='')
                sector = [ 0xf8 ]
                count = [128, 256, 512, 1024][id[3] & 0x03]
                count += 2   # for CRC
            sector.append(dt)
            count -= 1
            if count == 0:
                crc.reset()
                crc.data(sector)
                sector = sector[1:-2]   # remove DDAM and CRC
                if crc.get()==0:
                    if verbose:
                        print("CRC - OK")
                        dump_list_hex(_sector)
                    track.append([id, False, sector])
                else:
                    if verbose:
                        print("CRC - ERROR")
                id=[]
                state = State.IDLE    
        else:
            if verbose:
                print("**************WRONG STATE")
            state = State.IDLE
    return track


In [238]:
if test:
    track = decodeFormat(mfm_buf, mc_buf)
    print('{} sectors found.'.format(len(track)))
    for sec in track:
        print(sec[0])

48 sectors found.
[1, 1, 4, 1]
[1, 1, 5, 1]
[1, 1, 6, 1]
[1, 1, 7, 1]
[1, 1, 10, 1]
[1, 1, 11, 1]
[1, 1, 12, 1]
[1, 1, 13, 1]
[1, 1, 14, 1]
[1, 1, 15, 1]
[1, 1, 16, 1]
[1, 1, 2, 1]
[1, 1, 4, 1]
[1, 1, 5, 1]
[1, 1, 6, 1]
[1, 1, 7, 1]
[1, 1, 10, 1]
[1, 1, 11, 1]
[1, 1, 12, 1]
[1, 1, 13, 1]
[1, 1, 14, 1]
[1, 1, 15, 1]
[1, 1, 16, 1]
[1, 1, 2, 1]
[1, 1, 4, 1]
[1, 1, 5, 1]
[1, 1, 6, 1]
[1, 1, 7, 1]
[1, 1, 10, 1]
[1, 1, 11, 1]
[1, 1, 12, 1]
[1, 1, 13, 1]
[1, 1, 14, 1]
[1, 1, 15, 1]
[1, 1, 16, 1]
[1, 1, 2, 1]
[1, 1, 4, 1]
[1, 1, 5, 1]
[1, 1, 6, 1]
[1, 1, 7, 1]
[1, 1, 10, 1]
[1, 1, 11, 1]
[1, 1, 12, 1]
[1, 1, 13, 1]
[1, 1, 14, 1]
[1, 1, 15, 1]
[1, 1, 16, 1]
[1, 1, 2, 1]


In [94]:
#disk_data = 'fb30-00-0.txt'
#disk_data = 'protect_killer.txt'
disk_data = 'track10-4m-a.txt'
#disk_data = 'track2e-4m-a.txt'
#disk_data = 'track0-1m.txt'
#disk_data = 'fbasic30.txt'
disk_data = 'fb30a.txt'

disk = []
with open(disk_data, 'r') as f:
    for line in f:
        line = line.rstrip('\n')
        if '**TRACK_READ' in line:
            trk, side = [ int(v) for v in line.split(' ')[1:] ]
            print(trk, side, " ", end='')
            linebuf = ''
        elif '**TRACK_END' in line:
            if len(linebuf)==0:
                continue
            interval_buf = [ ord(c)-ord(' ') for c in linebuf ]

            """
            histo = generate_histogram(interval_buf)
            otsu = Otsu()
            otsu.generate_histogram(length)
            #otsu.display_histogram()
            otsu.threshold()
            th1 = otsu.get_optimal_threshold()
            otsu.mask_histogram(0, th1)
            otsu.threshold()
            th2 = otsu.get_optimal_threshold()
            print(th1, th2)
            th1, th2 = (20, 29)
            stream = restoreCD(interval_buf, th1, th2)
            """

            #"""
            stream = restoreCD2(interval_buf, clk_spd = 4e6, tracking_rate = 0.1)
            #stream = restoreCD2(interval_buf, clk_spd = 1e6, tracking_rate = 0.01)
            #"""
            mfm_buf, mc_buf = decodeMFM(stream)
            track = decodeFormat(mfm_buf, mc_buf, True)
            #print(track)
            disk.append(track)
        else:
            linebuf += line

10 0  IDAM CRC - OK
CHRN= [15, 1, 11, 1]
DAM CRC - OK
[ 0xFB, 0x01, 0x00, 0xED, 0x56, 0x16, 0x00, 0x20, 0xEC, 0x56, 0xC3, 0x02, 0x00, 0xED, 0x56, 0x16, 0x00, 0x16, 0x8C, 0x00, 0x00, 0x27, 0xDD, 0x8C, 0x00, 0x01, 0x27, 0xE2, 0x8C, 0x00, 0x02, 0x27, 0xE7, 0xEC, 0x56, 0xC3, 0x04, 0x00, 0xED, 0x56, 0x16, 0x00, 0xD9, 0xEC, 0x5E, 0xC3, 0x00, 0x16, 0xED, 0x5E, 0xEC, 0x5E, 0x34, 0x06, 0xCC, 0x00, 0x0C, 0x34, 0x06, 0x5F, 0x4F, 0x34, 0x06, 0x17, 0xF9, 0x4C, 0x32, 0x66, 0xEC, 0x5E, 0xC3, 0x00, 0x0C, 0xED, 0x5E, 0xAE, 0x5E, 0xCC, 0xF5, 0xF5, 0xED, 0x84, 0xEC, 0x5E, 0xC3, 0x00, 0x02, 0xED, 0x5E, 0xAE, 0x5E, 0x34, 0x10, 0xEC, 0x58, 0xC3, 0x00, 0x06, 0x1F, 0x01, 0xE6, 0x84, 0x4F, 0xC3, 0xF5, 0x00, 0xED, 0xF1, 0xEC, 0x5E, 0xC3, 0x00, 0x02, 0xED, 0x5E, 0xEC, 0x58, 0xC3, 0x00, 0x07, 0x1F, 0x01, 0xEC, 0x84, 0x83, 0x00, 0x01, 0xED, 0x54, 0x5F, 0x4F, 0x34, 0x06, 0xEC, 0x54, 0x35, 0x10, 0x34, 0x06, 0xAC, 0xE4, 0x10, 0x2E, 0x00, 0x19, 0xAF, 0x54, 0xEC, 0x5E, 0xE3, 0x54, 0x34, 0x06, 0xEC, 0x56, 0xE3, 0x54, 0x

[ 0xFB, 0x59, 0x20, 0x32, 0x2C, 0x58, 0x20, 0x20, 0x20, 0x41, 0x44, 0x44, 0x14, 0x4C, 0x02, 0x02, 0x27, 0xC2, 0x0C, 0x00, 0xF8, 0x38, 0x63, 0x23, 0x20, 0xE3, 0xE7, 0xE4, 0x27, 0xC8, 0xCE, 0x42, 0x02, 0x67, 0xC2, 0x0C, 0x0E, 0x02, 0x4C, 0xF8, 0x38, 0x63, 0x23, 0x20, 0x23, 0xE7, 0xE4, 0x27, 0xC8, 0xCC, 0xC1, 0x27, 0xC8, 0xCC, 0x08, 0x60, 0xC3, 0xCE, 0x02, 0x78, 0x38, 0x63, 0x23, 0x20, 0x63, 0xE7, 0xE4, 0x27, 0xC8, 0xCC, 0x0F, 0x00, 0x27, 0xC0, 0xC9, 0x00, 0xC8, 0xCC, 0x02, 0x0C, 0x3C, 0xE0, 0x27, 0x83, 0x86, 0x32, 0x32, 0x02, 0x3E, 0x7E, 0x42, 0x7C, 0x8C, 0xE4, 0x20, 0x26, 0x7C, 0x3C, 0x80, 0x00, 0x0F, 0x83, 0x86, 0x32, 0x32, 0x1E, 0x3E, 0x7E, 0x42, 0x7C, 0x8C, 0xCC, 0x12, 0x7E, 0x62, 0x4E, 0x3E, 0x3E, 0x3E, 0x13, 0x83, 0x86, 0x32, 0x32, 0x12, 0x3E, 0x7E, 0x42, 0x7C, 0x8C, 0xC0, 0xF0, 0x02, 0x7C, 0x84, 0x82, 0x32, 0x0C, 0x3C, 0xE0, 0x27, 0x83, 0x86, 0x32, 0x26, 0x3E, 0x3E, 0x7E, 0x42, 0x7C, 0x8C, 0xE4, 0x20, 0x26, 0x7C, 0x3C, 0x80, 0x00, 0x0F, 0x83, 0x86, 0x32, 0x26, 0x32, 0x3E, 0x7E, 0x

In [243]:
stream

'...'

In [249]:
print(len(mfm_buf))
otsu = Otsu()
otsu.generate_histogram(length)
otsu.display_histogram()

6294
0  :  0.0
1  :  0.0
2  :  0.0
3  :  0.0
4  :  0.0
5  :  0.0
6  :  0.0
7  :  0.0
8  :  1.0
9  :  3.0
10  :  5.0
11  :  3.0
12  :  5.0
13  :  7.0
14  :  183.0
15  :  4345.0
16  :  13772.0
17  :  3955.0
18  :  65.0
19  :  3.0
20  :  1.0
21  :  17.0
22  :  1222.0
23  :  3326.0
24  :  3134.0
25  :  4606.0
26  :  1475.0
27  :  68.0
28  :  16.0
29  :  0.0
30  :  32.0
31  :  1098.0
32  :  1126.0
33  :  752.0
34  :  274.0
35  :  48.0
36  :  12.0
37  :  9.0
38  :  2.0
39  :  1.0
40  :  2.0
41  :  2.0
42  :  2.0
43  :  0.0
44  :  0.0
45  :  0.0
46  :  0.0
47  :  1.0
48  :  1.0
49  :  0.0
50  :  1.0
51  :  0.0
52  :  1.0
53  :  1.0
54  :  0.0
55  :  0.0
56  :  0.0
57  :  0.0
58  :  0.0
59  :  1.0
60  :  0.0
61  :  0.0
62  :  0.0
63  :  1.0
64  :  0.0
65  :  1.0
66  :  0.0
67  :  0.0
68  :  0.0
69  :  0.0
70  :  0.0
71  :  0.0
72  :  0.0
73  :  0.0
74  :  0.0
75  :  0.0
76  :  0.0
77  :  0.0
78  :  0.0
79  :  0.0
80  :  0.0
81  :  0.0
82  :  0.0
83  :  0.0
84  :  0.0
85  :  0.0
86  :  0.0
87  

In [92]:
for track in disk:
    for sect in track:
        print(sect[0])

[15, 1, 11, 1]
[15, 1, 14, 1]
[15, 1, 3, 1]


In [91]:
for track in disk:
    for sect in track:
        for c in sect[2]:
            if c>=ord(' ') and c<=ord('Z'):
                print(chr(c), end='')

V VV'''VV44O4L24XOXTO4T54.TT4VTOT0 2X,VV=X,VV"X,VVV20D42454@C2.4.' 0.49.'4&2045A.4F.'42A45&T454@C2AXIXIXIXI44 5424545260 ' LBSR POUT1270 ' LDY #$00201280 ' LDU #$34001290 ' LBSR POUT1300 ' LDY #$00091310 ' LEAU NO3,PCR1320 ' LBSR POUT1330 ' LDY #$00201340 ' LDU #$35001350 ' LBSR POUT1360 ' LDY #$00091370 ' LEAU NO4,PCR1380 ' LBSR POUT1390 ' LDY 

In [89]:
dumpMFM(mfm_buf, mc_buf)

 8D 00 04 60 01 00 18 BF 44 00 00 CE FB C8 D0 0A 20 3F 00 F3 01 ED E8 95 0D 93 20 90 04 40 F0*0A
 6E 5E F7 40 20 05 E5 03 CF D9 C0 04 98 79 E3 E8 80 00 03 12 00 00 70 00 07 C4 75 48 08 AC 91 A9
 85 40 0F 98 18 0F F6 42 42 42*04 12 14*00 12 E4 E4 E4 E1*00 12 14*00 12 14*00 12 12 12 12 14*00
 E4 E1*00 E4 E9 09 12 70*02 C9 0A*00 E4 E9 09 09 0A*00 E4 E1*00 E4 E4 E4 E4 E1*00 12 E4 E5 09 09
 09 72 72 04 81 39 33 FF FF FF FF E0 00 3F FF 00 00 60 00 94*0A*A1 C7 00 00 82 FF 20 C9 40 90 97
 27 48 48 48*14 E4 E4 E4 E4 E4 E4 E1*00 12 E4 E4 E4 E1*00 12 14*00 1F FF FF FF FF FF FF FF FF FF
 00 00 00 0A*00 0A*00*A1 FB 16 0C 37 5E 64 65 9F 02 46 40 9E 00 15 7B 2F 6E 4F 14 65 D9 BC C3 B5
 08 AE B0 26 41 C1 D8 80 08 38 0C FB C1 80 42*02 63 60 9F A8 20 00 00 7F 1E 8A*00 3C 33 00 07 FF
 C1 00 30 1C 00 04 98 32 10 00 30 01 07 B0 FE CA 5A 70 66 33 16 7F 4F 00 00 48 00 00 00 43 FE 80
 0E 05 02 EB 9F 03 90 00 10 1C 54 10 00 45 40 32 31 CC 01 1C 48 0E C0 DD F5 F5 98 83 C3 C0 40 00
 02 3E 7F 8B 73 4E 48 00 6C 38

 25 2A 5E 84*0A*02 5F E5 7A 59 96 9B DF 3C 80 78 78 93 50*0A E7 66 05 E6 BA BC*02 03 7C EA 85 F5
 65 D5 00*14 F0*14 00 92 10 40 06 3C 20 E0 04 24 24 24 24 24 24 24 09 D2 12 12 12 12 12 12 04 E4
 09 09 09 09 02 72 72 72 72 72 74 84 84 84 84 84 84 84 84 89 39 39 38*02 CA 72 72 05*00 E4 E4 E1
*00 E4 E4 E9 72 70 3F FF FF FF FF FF FF FF F0 00 00 02 68*01 87 F8 7F F9 D8 03 76 C0 9C 9C 9C 9C
 9C 81 21 2E*02 12 12 12 12 12 14*00 12 14*00 E4 E9 72 72 72 70 C0 00 00 00 00 00 1F FE 3F FF FF
 FF FE*02*02*A1*A1 F0 02 08 40 40 00 00 5E 4D 9F 81 01 76 C0 11 00 6C 72 BD 95 56 5C 01*02 F0 06
 0F 53 ED 50 40 4C 00 04 2F BA 82 48 00 09 02 63 90 20 E0 00 C9 07 00 80 08 08 09*21 87 AA 25 97
 BA 40 80 00 88 60 00 09 08 30*02 B0 B7 C9 00 40 36 03 18 20 8A 08*14 66 C8 10 86 06 00 06 60 00
 12 D3 B2 2C 00 80 17 27 D4 E8 7B 2A 7A 22 11 00 00 09 19 C0 F9 00 37 D3 52 54 80 61 C0 00 55 C0
 0B CD 77 22 89 14 4F 02*14 0F 60 60 9D 65 07 D8 8E 87 30 9A*01 19 01 81 38 38 06 20 C8 06 83 00
 32 15 E9*42 F2 52 0C 08 0A D7

 7F FE 7C FE 23 B7 A7 20*14 E4 E4 09 09 02 04 84 84 84 85*00 12 12 E1*00 E4 E4 09 09 02 72 70 3F
 FF FF F0 00 00 00 00 00 00 01 80 01*04 0A*00 A4 00 00 04 20*14*14 00 73 E1*00 01 6E A7 BE 7F 10
 40 00 7D*01 04 00 01 E6 00 00 E0 01 00 08 7C 40 D7 5C 01 F9 84 3B CE 0D E0 26 8C*02 01 E2 60 3C
 0E C8 5F FE 70 08 03 A1 38 B8 78*14*02 08 44 01 B3 90 41 BE 73 A8 7E 05 FD F8 04 8A*00 08 63 E1
 17 62 00 00 DC 68 00 00 00 22 26 A1 24 58 00 09 00 32 04 99 F8 E6 D1 9A 19 02 39 E0 04 33 88 60
 D9*42 00 6E 07 C0*0A 08 00 4D 81 31 FF E7 D5 01 88 8C 8C 04 C8 99 9B 2D 48 FB 8B 00 2E DB 54 80
 7B D5 C0 0E 20 08 D2 1F FF 00 2B 30 00 C5 F8 60 07 C1 6E 65 30 24 27 21 82 48 FC 00 13 C0 74 0C
 01*04 71 30 1C 31 44 B8 1F C0 24 00 02 31 31 37 B1 72 EC F4 A4 20 18 3F C0 0C 8A 9D B4 07 8F 4A
 C4 05*00 E7 EE A9 41 50 58 54 50 41*42 44 24 25 51 44 68 30*02 55 49 98 62*04 48 20*14 33 8A*00
 10 01 A8 27 AC 03 CC 56 53 53 52 55 30 79 D8 A9 A4 40 08 21 21 3B 05*00 0A E2 AF FE 21*04 12 04
 E4 E4 E4 E4 E1*00 12 14*00 E4

 3F C2 00 17 D6 01 78 10 42 0F 86 01 3B 2F C8 A0 3C 40 01 00 1C A3 A0 04 13 80 00 41*04 00 00 04
 00 0C A1 AE AD 1A 10 09 0C 80 21 38 65 8A D4*14 08 C2 BE 9E BE 84 20 F0 60 4C 03 C1 D4 FE 20 70
 23 C1 E6 89 F7 AC 3F 81 5A*01 10 04 66 CE 3B C1 9E F6 08 73 29 41 87 8E 01 A0 0E 9E 0F EF C3 30
 05 90 58 60 00 3E 18 03 B3 0C F9 D1 B3 81 00 01 F8 24 00 80 7A 75 C0 B6 CF 19 49 48 44 1D E3 DE
 D0*01 C1 08 00 C6 4B 38 8E C0 F8 75 42 E6 01 24 D0 20 70*02*02 21*04 B5 18 30 0F AB E3 57 01 79
 C8*14 C0 0F 6C 30 0D F7 A6 2D FD 38 E3 A0 10 04 41 E0 E2 FB 65 0C F0 41 20 30 4D 00 40 A1 35 B5
 05 C9 D1 20 40 8F 0E D2 66 80 10 41 20 21 D3 23 7C 4C A8 13 48 40*0A 5A 28 E0 56 A6 50 01 02 0E
 00 1C 13 00 37 21 DF CE 4E 40 90 97 27 48 48 48 4B*42 12 E4 E4 E4 E1*00 12 14*00 12 E4 E4 E4 E4
 E4 0A*00 12 E4 E4 E4 E4 E4 09 09 72 74 84 84 85*00 E4 E4 09 02 72 64 84 B8*02 12 E4 E4 E4 E4 E4
 E0 00 07 FF E0 00 00 00 00 00 00 FF F2 9A*01 87 F8 00 01 D6 02*04 C0 01 72 72 72 72 05*00 12 E1
*00 E4 E9 72 72 72 72 72 72 70

 84 81 3A 5C 9C 9C 9D 20 40 90 90 27*21 D2 12 12 05*00 D0*01 C9 C9 D0*01 D0*01 C9 C9 C9 D0*01 C9
 C9 C9 CA 12 12 12 12 12 04 09 09 02*02 C9 C9 D0*01 C9 C9 D0*01 C9 C9 C9 C9 D2 12 05*00 C9 CA 04
 E4 82 72 72 72 74 84 81*02 C9 CA 12 12 64 E4 E4 E4 E9 09 02*02 C9 CA 12 12 12 04 E4 E8*01 C9 CA
 12 12 64 E4 E4 E8*01 C9 CA 12 12 12 12 12 05*00 C9 D0*01 C9 C9 C9 C9 C9 C9 C9 CA 04 E4 E9 09 09
 09 02 04 81*02 D2 12 05*00 C9 C9 C9 C9 C9 C9 D2 E4 E5 02 72 74*01 C9 C9 C9 C9 C9 C9 D0 50 96 42
*04 12 14*00 E4 E4 09 08 72 72 72 70*02 E4 E4 09 09 09 0A*00 12 14*00 12 12 12 12 12 12 14*00 12
 14*00 12 00 01 80 03 00 06 00 03 00 06 00 30 00 42 0A*00 87 F8 01 E2 02 FC 22 F1 39 38*02 E4 E1
*00 12 12 12 12 E4 E4 09 09 09 09 0A*00 12 E4 E4 E4 E4 E1*00 E0 0F FF FF FF FF FF FF FF FF FF FF
 FF F0*02*02 0A*00 A7 ED 5D 95 B1 30 00 00 25 62 83 93 7B A3 00 60 31 C0 10 37 32 14*00 05 58 38
 20 44 F8 20 19 03 A3 43 29 00 40 01 D1 C9 85 B9 8D 81 3E 06 01 40 80 20 0F 04 38*02 7A D2 02 08
 19 F3 90 00 3A 01 24 F8 C0 48

In [39]:
a=[0xA1, 0xA1, 0xA1, 0xFB, 
   0xAB, 0x8D, 0x53, 0x8E, 0x71, 0xBA, 0x8D, 0x4E, 0x8D, 0x4F, 0xBD, 0xDB, 0x49, 0xBD, 0xD8, 0x07,
   0x25, 0xEA, 0x9F, 0xD9, 0xC6, 0x02, 0x6D, 0x01, 0x27, 0x04, 0x8D, 0x40, 0x25, 0xDE, 0x5A, 0x2B, 
   0xDB, 0xC1, 0x03, 0x22, 0xD7, 0xF7, 0x71, 0xFA, 0x8D, 0x2F, 0x8E, 0x71, 0xAB, 0x8D, 0x27, 0x8E,
   0x71, 0xC8, 0x8D, 0x22, 0x8D, 0x23, 0xBD, 0xDB, 0x49, 0xBD, 0xD8, 0x07, 0x25, 0xEA, 0x9F, 0xD9,
   0xC6, 0x01, 0x6D, 0x01, 0x27, 0x04, 0x8D, 0x14, 0x25, 0xDE, 0x5D, 0x2B, 0xDB, 0xC1, 0x0F, 0x22,
   0xD7, 0xF7, 0x71, 0xFB, 0x20, 0x2E, 0x7E, 0x74, 0xA6, 0x7E, 0xD9, 0x2F, 0x5F, 0x34, 0x06, 0x9D,
   0xD2, 0x26, 0x05, 0x00, 0x03, 0xC7, 0x08, 0x06, 0x10, 0x01, 0x03, 0xC6, 0x43, 0x21, 0xFE, 0x38,
   0x02, 0x02, 0x42, 0x23, 0x86, 0x3C, 0x30, 0x00, 0x02, 0x4C, 0x9F, 0xC2, 0x03, 0x42, 0x09, 0x01,
   0xF8, 0x0F, 0x1C, 0x1F, 0xC8, 0x48, 0x03, 0xE0, 0x00, 0xC0, 0x00, 0x10, 0xC0, 0x18, 0x82, 0x4F,
   0xE4, 0x10, 0x18, 0x00, 0x40, 0x00, 0xC0, 0x78, 0x82, 0x4F, 0xCE, 0x06, 0x10, 0xC0, 0x00, 0x10,
   0xC0, 0x00, 0x07, 0xC8, 0x82, 0x4F, 0xCE, 0x00, 0x18, 0x00, 0x08, 0x83, 0xC0, 0x08, 0x86, 0x10,
   0xC4, 0x82, 0x1F, 0xC0, 0x43, 0x1E, 0x00, 0x07, 0x04, 0x00, 0x10, 0xC0, 0x67, 0x13, 0xE1, 0x00,
   0x0E, 0x44, 0x7F, 0x9F, 0xC0, 0x1E, 0x11, 0x9C, 0x19, 0xC0, 0x1C, 0x8E, 0x20, 0x00, 0x1F, 0x30,
   0x4C, 0x20, 0x19, 0x00, 0x1F, 0x30, 0x00, 0x1F, 0x20, 0x0C, 0x20, 0x01, 0xC0, 0x1F, 0x21, 0xC0,
   0x1F, 0x27, 0x0C, 0x20, 0x92, 0x40, 0x1F, 0x27, 0x00, 0x1F, 0x30, 0x0C, 0x20, 0xC0, 0x00, 0x1F,
   0x27, 0xC0, 0x1F, 0x26, 0x0C, 0x20, 0x80, 0x80, 0x1F, 0x26, 0x00, 0x1F, 0x24, 0xCC, 0x20, 0x00,
   0xC0, 0x1F]
b = [ 0xA1, 0xA1, 0xA1, 0xFE, 0x00, 0x01, 0x01, 0x01, 0xCD, 0x3C ]
c = [                   0xFE, 0x00, 0x01, 0x01, 0x01, 0xCD, 0x3C ]
d = [ 0xA1, 0xA1, 0xA1, 0xFE, 0x00, 0x01, 0x01, 0x01, 0x00, 0x00 ]
e = [                   0xFE, 0x14, 0x00, 0x01, 0x01, 0x2B, 0x5A ]
crc = CCITT_CRC()
#crc.reset()
crc.reset()
crc.data(f)
print(format(crc.get(), '04X'))

CDCF
